In [0]:
from pyspark.sql.functions import col, when, lit, concat, trim, count,year,substring,desc, coalesce, array, desc_nulls_last, row_number, date_format, current_timestamp, date_add
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.types import StringType
from datetime import *

In [0]:
df_compra = spark.table("context_site.compra").select(col("idcompra"), col("data").alias("data_compra"), "flagaprovado", "idunidadenegocio","bandeira")           
df_compra_cliente = spark.table("context_site.compracliente").select(col("nome").alias("nome_cliente"), "sobrenome", col("telefoneresidencial").alias("telefone_residencial"), col("telefonecelular").alias("telefone_celular"), "email", "cpfcnpj", col("idcompra"), "tipocliente", "bandeira").filter(col("tipocliente") == "0")

df_unidade_negocio = spark.table("context_site.unidadenegocio").select(col("nome").alias("nome_unidade_negocio"), "idunidadenegocio", "bandeira")

df_compra_entrega_status_estorno = spark.table("context_site.compraentregastatusestorno").select("status", "dataultimaatualizacao", "idcompra", "bandeira")
df_compra_entrega_status_estorno = df_compra_entrega_status_estorno.withColumnRenamed("status", "status_estorno")

df_compra_entrega = spark.table("context_site.compraentrega").select("gerencialid","idcompraentrega", "idcompraentregastatus", "origem", col("idcompra"), "bandeira").filter(col("origem") == "TD")
df_compra_entrega = df_compra_entrega.withColumnRenamed("idcompraentrega", "idcompraentreganova")
df_compra_entrega_estorno = spark.table("context_site.compraentregaestorno").selectExpr("valor", "idcompraentreganova", "tipopagamento", "bandeira")
df_compra_entrega_estorno = df_compra_entrega_estorno.withColumnRenamed("tipopagamento", "idtipopagamento")

df_forma_pagamento = spark.table("context_site.formapagamento").selectExpr("nome as nome_forma_pagamento", "idformapagamento", "tipo as tipo_forma_pagamento", "bandeira")

df_liberar_entrega = spark.table("context_site.liberarentrega").select("gerencialid", "status", "bandeira")
df_liberar_entrega = df_liberar_entrega.withColumnRenamed("status", "status_liberar_entrega")

df_compra_forma_pagamento = spark.table("context_site.compraformapagamento").select(col("idcompra"), col('idformapagamento'), "bandeira")

#CompraEntregaStatusEstorno
df_motor_sap = spark.table("app_atendimento.tb_motor_sap").select("forma_restituicao", "idcredito", col("data_criacao_credito").alias("data_criacao_credito_motor"), col("data_atualizacao_status").alias("data_atualizacao_status_motor"), col("status").alias("status_motor"), col("unidade_negocio"), col("aging").alias("aging_motor"), col("faixa_aging").alias("faixa_aging_motor"), col("status_finalizador").alias("status_finalizador_motor"), col("area_finalizadora").alias("area_finalizadora_motor"),"pedido", col("direcionamento").alias("direcionamento_motor"), "forma_pagamento", "entrega")
df_motor_sap = df_motor_sap.withColumnRenamed("pedido","idcompra")

In [0]:
df_compra = spark.table("context_site.compra").select(col("idcompra"), col("data").alias("data_compra"), "flagaprovado", "idunidadenegocio","bandeira")           
df_compra_cliente = spark.table("context_site.compracliente").select(col("nome").alias("nome_cliente"), "sobrenome", col("telefoneresidencial").alias("telefone_residencial"), col("telefonecelular").alias("telefone_celular"), "email", "cpfcnpj", col("idcompra"), "tipocliente", "bandeira").filter(col("tipocliente") == "0")

df_unidade_negocio = spark.table("context_site.unidadenegocio").select(col("nome").alias("nome_unidade_negocio"), "idunidadenegocio", "bandeira")

df_compra_entrega_status_estorno = spark.table("context_site.compraentregastatusestorno").select("status", "dataultimaatualizacao", "idcompra", "bandeira")
df_compra_entrega_status_estorno = df_compra_entrega_status_estorno.withColumnRenamed("status", "status_estorno")

df_compra_entrega = spark.table("context_site.compraentrega").select("gerencialid","idcompraentrega", "idcompraentregastatus", "origem", col("idcompra"), "bandeira").filter(col("origem") == "TD")
df_compra_entrega = df_compra_entrega.withColumnRenamed("idcompraentrega", "idcompraentreganova")
df_compra_entrega_estorno = spark.table("context_site.compraentregaestorno").selectExpr("valor", "idcompraentreganova", "tipopagamento", "bandeira")
df_compra_entrega_estorno = df_compra_entrega_estorno.withColumnRenamed("tipopagamento", "idtipopagamento")

df_forma_pagamento = spark.table("context_site.formapagamento").selectExpr("nome as nome_forma_pagamento", "idformapagamento", "tipo as tipo_forma_pagamento", "bandeira")

df_liberar_entrega = spark.table("context_site.liberarentrega").select("gerencialid", "status", "bandeira")
df_liberar_entrega = df_liberar_entrega.withColumnRenamed("status", "status_liberar_entrega")

df_compra_forma_pagamento = spark.table("context_site.compraformapagamento").select(col("idcompra"), col('idformapagamento'), "bandeira")

#CompraEntregaStatusEstorno
df_motor_sap = spark.table("app_atendimento.tb_motor_sap").select("forma_restituicao", "idcredito", col("data_criacao_credito").alias("data_criacao_credito_motor"), col("data_atualizacao_status").alias("data_atualizacao_status_motor"), col("status").alias("status_motor"), col("unidade_negocio"), col("aging").alias("aging_motor"), col("faixa_aging").alias("faixa_aging_motor"), col("status_finalizador").alias("status_finalizador_motor"), col("area_finalizadora").alias("area_finalizadora_motor"),"pedido", col("direcionamento").alias("direcionamento_motor"), "forma_pagamento", "entrega")
df_motor_sap = df_motor_sap.withColumnRenamed("pedido","idcompra")

In [0]:
df_incidents  = spark.table("context_atendimento.incidents").select("fk_pedido", col("lookupname").alias("protocolo"), "statustype", "queue", col("id").alias("identificacao_do_incidente"), "product", "category", col("createdtime").alias("data_de_criacao"), "contact", "tipo_incidente", "banco", "agencia", "digito_agencia", "conta", "digito_conta", "disposition")
df_tipo = spark.table("context_atendimento.co_tipo").select("name", "id")

df_pedido = spark.table("context_atendimento.pedido").select("id", col("numero_pedido").alias("idcompra"), "unidade_negocio")

df_entrega = spark.table("context_atendimento.entrega").select("fk_pedido", col("numero_entrega").alias("gerencialid"))

df_unidade_negocio = spark.table("context_atendimento.unidade_negocio").select(col("nome").alias("unineg"), "id")

df_status_type = spark.table("context_atendimento.status_type").select("id", col("name").alias("status_rn")).filter(col("status_rn").isin("Não Resolvido", "Atualizado", "Aguardando", "Aguardando retorno do cliente", "Aberto", "Atendimento agendado", "Atendimento cancelado", "Atendimento Crítico", "Insucesso de contato", "Orientação de uso", "Necessário informar o órgão da tratativa", "Aguardando MCE"))

df_queue = spark.table("context_atendimento.queue").select("id", col("name").alias("fila"))

df_products = spark.table("app_atendimento.srvco_prdto").select(col("nm_prdto").alias("produto_nivel_03"), col("nm_prdto_nivel_02").alias("produto_nivel_02"), col("nm_prdto_nivel_03").alias("produto_nivel_01"), "cd_prdto")

#df_products = (df_products.filter((col("produto_nivel_01") == "Reativo") & (col("produto_nivel_02").isin("Correção de dados", "Financeiro"))
#                                 |(col("produto_nivel_01") == "Solicitação") & (col("produto_nivel_02") == "Financeiro")
#                                 |(col("produto_nivel_01") == "Preventivo") & (col("produto_nivel_02").isin("Correção de dados", "Financeiro"))))

df_products = (df_products.filter((col("produto_nivel_01").isin("Correção de dados", "Financeiro")) 
                                 | (col("produto_nivel_02").isin("Correção de dados", "Financeiro"))
                                 |(col("produto_nivel_03").isin("Correção de dados", "Financeiro"))))
                                 

df_categoria = spark.table("app_atendimento.srvco_ctgra").select(col("nm_ctgra_nivel_02").alias("categoria_nivel_02"), col("nm_ctgra_nivel_03").alias("categoria_nivel_01"), col("nm_ctgra").alias("categoria_nivel_03"), "cd_ctgra")
df_contact = spark.table("context_atendimento.contact").select("cpf_cnpj", "id", "email", "nome_completo")
df_dispositions = spark.table("app_atendimento.srvco_dispo").select("nm_dispo", "cd_dispo")

In [0]:
df_rn = (df_incidents.join(df_pedido,  df_incidents.fk_pedido == df_pedido.id, 'inner')
                          .join(df_entrega, df_pedido.id == df_entrega.fk_pedido, 'inner')
                          .join(df_status_type, df_incidents.statustype == df_status_type.id, 'inner')
                          .join(df_queue, df_queue.id == df_incidents.queue, "left")
                          .join(df_unidade_negocio, df_pedido.unidade_negocio == df_unidade_negocio.id, "inner")
                          .join(df_products, df_incidents.product == df_products.cd_prdto, "inner")
                          .join(df_categoria, df_incidents.category == df_categoria.cd_ctgra, "inner")
                          .join(df_contact, df_incidents.contact == df_contact.id, "inner")
                          .join(df_tipo,  df_incidents.tipo_incidente == df_tipo.id, 'inner')
                          .join(df_dispositions, df_incidents.disposition == df_dispositions.cd_dispo, "left")
                          .withColumn("banco", when(col("banco").isNull(), lit("Nenhum Valor")).otherwise(col("banco")))
                          .withColumn("agencia", when(col("agencia").isNull(), lit("Nenhum Valor")).otherwise(col("agencia")))
                          .withColumn("digito_agencia", when(col("digito_agencia").isNull(), lit("Nenhum Valor")).otherwise(col("digito_agencia")))
                          .withColumn("conta", when(col("conta").isNull(), lit("Nenhum Valor")).otherwise(col("conta")))
                          .withColumn("data_de_criacao", when(col("data_de_criacao").isNull(), "Nenhum Valor").otherwise(date_format('data_de_criacao', 'MM/dd/yyy                                                 HH:MM').alias('data_de_criacao')))
                          .withColumn("digito_conta", when(col("digito_conta").isNull(), lit("Nenhum Valor")).otherwise(col("digito_conta")))
                          .withColumn("fila", when(col("fila").isNull(), lit("Nenhum Valor")).otherwise(col("fila")))
                          .select("protocolo", "identificacao_do_incidente", "unineg", "data_de_criacao","idcompra",col("email").alias("email_rn"), "gerencialid", "banco", "agencia", "digito_agencia",                                         "conta", "digito_conta", "status_rn","fila",df_tipo.name.alias("tipo_incidente"),"nome_completo", "cpf_cnpj","categoria_nivel_01","categoria_nivel_02",                                                     "categoria_nivel_03", "produto_nivel_01","produto_nivel_02", "produto_nivel_03"))

In [0]:
df_rn = (df_rn.withColumn("categoria_nivel_1", when(col("categoria_nivel_01").isNull(), col("categoria_nivel_02")).otherwise(col("categoria_nivel_01")))
                       .withColumn("categoria_nivel_2", when(col("categoria_nivel_01").isNull(), col("categoria_nivel_03")).otherwise(col("categoria_nivel_02")))
                       .withColumn("categoria_nivel_3", when(col("categoria_nivel_01").isNull(), lit("Nenhum Valor")).otherwise(col("categoria_nivel_03")))
                       .drop("categoria_nivel_01")
                       .drop("categoria_nivel_02")
                       .drop("categoria_nivel_03"))
  

In [0]:
df_rn = df_rn.withColumn('row_number', row_number().over(Window.partitionBy("protocolo").orderBy(desc("protocolo"))))

df_rn = df_rn.filter(col('row_number') == 1).drop('row_number')

In [0]:
df_rn = df_rn.withColumn("origem_pedido",
                         when(col("idcompra") == 0 ,"Sem pedido informado")
                        .when(col("idcompra").between(1, 100000000), "SIGE")
                        .when(col("idcompra").between(100000001, 166999999), "LN")
                        .when(col("idcompra") > 167000000, "Legado"))

In [0]:
df_not_motor = df_rn.join(df_motor_sap.withColumn("gerencialid", col("entrega")),["idcompra"], how='leftanti')

In [0]:
#3004
df_not_motor_front = (df_not_motor.join(df_compra_entrega, ['gerencialid'], how='inner')
#2990  
                    .join(df_compra, (df_compra.idcompra == df_compra_entrega.idcompra) & 
                                     (df_compra.bandeira == df_compra_entrega.bandeira), how='inner')
                    .join(df_compra_forma_pagamento, (df_compra.idcompra == df_compra_forma_pagamento.idcompra) & 
                                                     (df_compra.bandeira == df_compra_forma_pagamento.bandeira), how='inner')
#3470
                    .join(df_forma_pagamento, (df_compra_forma_pagamento.idformapagamento == df_forma_pagamento.idformapagamento) &
                                              (df_compra_forma_pagamento.bandeira == df_forma_pagamento.bandeira), how='inner')
#3470
                    .join(df_compra_cliente, (df_compra_cliente.idcompra == df_compra.idcompra)  &
                                             (df_compra_cliente.bandeira == df_compra.bandeira), how='inner')
#3453
                    .join(df_unidade_negocio, (df_unidade_negocio.id == df_compra.idunidadenegocio))
#3453
                    .join(df_compra_entrega_estorno, (df_compra_entrega_estorno.idcompraentreganova == df_compra_entrega.idcompraentreganova) &                                                          (df_compra_entrega_estorno.bandeira == df_compra_entrega.bandeira), how='left')
                    .join(df_liberar_entrega, ['gerencialid', 'bandeira'], how='left')
                    .join(df_compra_entrega_status_estorno, ['idcompra', 'bandeira'], how='left')
                    .drop(df_compra_forma_pagamento.idformapagamento)
                    .drop(df_compra_entrega.idcompra)
                    .drop(df_compra_forma_pagamento.idcompra)
                    .drop(df_compra_cliente.idcompra)
                    .drop(df_compra.idcompra))

In [0]:
df_not_motor_front_final = (df_not_motor_front.withColumn("forma_restituicao", lit(""))
                            .withColumn("idcredito", lit(""))
                            .withColumn("data_criacao_credito_motor", lit(""))
                            .withColumn("data_atualizacao_status_motor", lit(""))
                            .withColumn("status_motor", lit(""))
                            .withColumn("unidade_negocio", lit(""))
                            .withColumn("aging_motor", lit(""))
                            .withColumn("faixa_aging_motor", lit(""))
                            .withColumn("status_finalizador_motor", lit(""))
                            .withColumn("area_finalizadora_motor", lit (""))
                            .withColumn("forma_pagamento", lit("")))

In [0]:
df_not_motor_front_final = df_not_motor_front_final.dropDuplicates()

In [0]:
df_in_motor = df_rn.join(df_motor_sap, ["idcompra"], how='inner')

In [0]:
cond1 = col("nome_forma_pagamento") == "Vale"
cond2 = col("flagaprovado") == False
df_not_motor_front_final = (df_not_motor_front_final.withColumn("direcionamento_motor",
                                        when(col("nome_forma_pagamento").like("%B2B%") , "Atendimento - Verificar Estorno de Pontos")
                                       .when(col("nome_forma_pagamento").like("%Carnê Digital%") , "Carnê Digital")
                                       .when(col("idcompra").between(1,100000000), "Atendimento - Pedido SIGE - Verificar Reclamacao")
                                       .when(col("idcompraentregastatus") == "CIC", "Atendimento - Insucesso de coleta Solicitar Coleta Manual a CRL")
                                       .when(col("idcompraentregastatus") == "RCL", "Atendimento - Cliente recusou a coleta")
                                       .when(col("flagaprovado") == False , "Venda Reprovada pelo Proxy")
                                       .when(((cond1) & (cond2)), "TI - Liberar Vale")
                                       .when(col("idcompraentregastatus") == "ENT", "Entrega Realizada")
                                       .when(col("idcompraentregastatus") == "_CANM", "Entrega Cancelada")
                                       .when(col("idcompraentregastatus") == "AS6", "TI - Liberar Reenvio")
                                       .when(col("idcompraentregastatus") == "WMS", "Pedido em Separação de Estoque")
                                       .when(col("idcompraentregastatus") == "ARP", "Compra Cancelada Antes da Aprovação do Pgto")
                                       .when(col("idcompraentregastatus") == "AMC", "Aguardando análise de crédito")
                                       .when(col("idcompraentregastatus") == "_TMA", "Atendimento - Pedido já Atendido por TMA")
                                       .when(col("idcompraentregastatus") == "_ADV", "TI - Reenviar Entrega")
                                       .when(col("idcompraentregastatus") == "_RCE", "TI - Analisar RCE")
                                       .when(col("idcompraentregastatus") == "_EPE", "TI - Pedido com Erro de Integração")
                                       .when(col("idcompraentregastatus") == "_EVE", "TI - Pedido com Erro de Integração")
                                       .when(col("idcompraentregastatus") == "_PAP", "TI - Pedido com Erro de Integração")
                                       .when(col("idcompraentregastatus") == "DVC", "TI - Liberar Restituição")
                                       .when(col("status_liberar_entrega").isin(3,6), "TI - Realizar o Envio do IF")
                                       .when((col("origem_pedido") == "LN") &
                                             (col("idcompraentregastatus") == "MIG"), "TI - Verificar Status da Migração do Pedido")
                                       .when((col("origem_pedido") == "LN") &
                                             (col("status_estorno") != "Confirmado"), "Atendimento - Verificar com CSC")
                                       .when((col("status_estorno") == "Confirmado") &
                                             (col("origem_pedido") == "LN"), "Restituido no LN")
                                       .when(col("idcompra") == 0, "Sem pedido informado")
                                       .otherwise("Não se Trata de Restituição")))

In [0]:
df_in_motor_front = (df_in_motor.withColumn("nome_cliente", lit(""))
                                .withColumn("sobrenome", lit(""))
                                .withColumn("telefone_residencial", lit(""))
                                .withColumn("telefone_celular", lit(""))
                                .withColumn("email", lit(""))
                                .withColumn("cpfcnpj", lit(""))
                                .withColumn("tipocliente", lit('')))

In [0]:
colunas = df_rn.drop(*["idcompra","unineg"]).columns
colunas.extend(df_motor_sap.drop("entrega").columns)

colunas.extend(df_compra_cliente.drop("idcompra", "bandeira").columns)

In [0]:
df_backlog =  df_in_motor_front.select(colunas).unionByName(df_not_motor_front_final.select(colunas))

In [0]:
WindowSpec = Window.partitionBy("protocolo")
df_backlog = df_backlog.withColumn("qtd_protocolo" ,
                                count("protocolo").over(WindowSpec))

In [0]:
df_backlog = df_backlog.withColumn("qtd_forma" , when(col("qtd_protocolo") > 1, ">1 Forma").otherwise("1 Forma"))

In [0]:
df_backlog2 = df_backlog.withColumn("dt_processamento", date_format(current_timestamp(),"yyyy-MM-dd"))\
                         .withColumn("data_minima", date_add(col("dt_processamento"),-5))
df_backlog2 = df_backlog2.filter(col("dt_processamento") >= date_add(current_timestamp(),-5))

In [0]:
%run MotorIngestao/RAW/DW_Connector

In [0]:
writeDW(spark, df_backlog2, 'gestao_cliente.backlog_rest', mode='append', additional_options=None)